In [15]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, TimestampType
import numpy as np
from scipy import sparse
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import pandas as pd
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import warnings
import seaborn as sns
import cv2

You should consider upgrading via the 'C:\Users\AndréUser\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'cv2'

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.master("local[*]").appName('TCC').getOrCreate()
sc = spark.sparkContext

In [3]:
#Carregamento do arquivo com inferencia do Schema e header
df = spark.read.csv('project/CSVs/basics.csv', inferSchema=True, header=True)

In [4]:
df_basic = df.dropDuplicates()

In [5]:
df_basic = df_basic.filter("publishedAt <= '2022-11-30 23:59:59'")

In [5]:
df_comments_per_user = df_basic.groupby("authorChannelId").count()

In [8]:
df_high_commenters= df_comments_per_user.filter("count>500").withColumnRenamed("authorChannelId", 'authorChannelId_HC')

In [9]:
df_high_commenters.count()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [9]:
high_commenters_dates = df_high_commenters[['authorChannelId_HC']].join(df_basic[['authorChannelId', 'publishedAt']], df_high_commenters.authorChannelId_HC == df_basic.authorChannelId, 'left').drop('authorChannelId_HC')

In [10]:
dates_as_arrays = high_commenters_dates.groupby("authorChannelId").agg(collect_list("publishedAt"))

In [11]:
dates_as_arrays.columns

['authorChannelId', 'collect_list(publishedAt)']

In [16]:
# definir o formato de data e hora
fmt = '%Y-%m-%d %H:%M:%S'

# definir o início e o fim do período
d1 = datetime.strptime('2022-09-01 00:00:01', fmt)
d2 = datetime.strptime('2022-11-30 23:59:59', fmt)

# calcular a diferença em dias
delta = d2 - d1
days = delta.days + 1  # +1 para incluir o último dia

# número de segundos em um dia
seconds_per_day = 24 * 60 * 60

# criar uma matriz esparsa com zeros
sparse_matrix = sparse.dok_matrix((days, seconds_per_day), dtype=np.int8)

In [17]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from PIL import Image
# Define the schema for the DataFrame
schema = StructType([
    StructField("authorChannelId", StringType(), True),
    StructField("imgNumber", IntegerType(), True)
])

data = []

for x,y in dates_as_arrays.toPandas().iterrows():
  # criar uma matriz esparsa com zeros
  sparse_matrix = sparse.dok_matrix((days, seconds_per_day), dtype=np.int8)
  author = y['authorChannelId']
  data.append((author, x))
  for d in y['collect_list(publishedAt)']:
    day = (d -d1).days
    seconds = d.hour*3600 + d.minute*60 + d.second
    sparse_matrix[day-1,seconds-1] +=1
  dense_matrix = sparse_matrix.toarray()
  # Get the non-zero entries of the sparse matrix
  nonzero_rows, nonzero_cols = sparse_matrix.nonzero()
  # Scatter plot the non-zero entries
  #plt.imshow(dense_matrix, cmap='gray')
  plt.xlim(-10000,100000)
  plt.ylim(-10, 100)
  plt.scatter(nonzero_cols, nonzero_rows, marker='s', s=80, alpha=0.2, color='red')
  plt.axis('off')
  plt.savefig(f'project//IMGs//red_scatter//{x}.png')
  plt.clf()
  #plt.show()
df = spark.createDataFrame(data, schema)
df.write.csv("project//IMGs//image_map.csv", header=True, sep=";")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\AndréUser\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\agrri\AppData\Local\Temp\ipykernel_30484\2109151119.py", line 11, in <module>
    for x,y in dates_as_arrays.toPandas().iterrows():
  File "C:\Program Files\spark\spark-3.4.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py", line 208, in toPandas
    pdf = pd.DataFrame.from_records(self.collect(), columns=self.columns)
  File "C:\Program Files\spark\spark-3.4.1-bin-hadoop3\python\pyspark\sql\dataframe.py", line 1216, in collect
    sock_info = self._jdf.collectToPython()
  File "C:\Program Files\spark\spark-3.4.1-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "C:\Program Files\spark\spark-3.4.1-bin-hadoop3\p

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
# load the first image
image1 = cv2.imread('/content/drive/My Drive/TCC/images_red/0.png')

# path to the directory containing the other images
directory = '/content/drive/My Drive/TCC/images_red/'

df_mse = pd.DataFrame(columns=['Image1', 'Image2', 'MSE'])
df_nrmse = pd.DataFrame(columns=['Image1', 'Image2', 'NRMSE'])
df_ssim = pd.DataFrame(columns=['Image1', 'Image2', 'SSIM'])
df_psnr = pd.DataFrame(columns=['Image1', 'Image2', 'PSNR'])

images = [f for f in os.listdir(directory) if f.endswith('.png')]  # assuming the images are in png format

for i in range(len(images)):
    for j in range(i+1, len(images)):
        print(i,j)
        img1_path = os.path.join(directory, images[i])
        img2_path = os.path.join(directory, images[j])

        image1 = cv2.imread(img1_path)
        image2 = cv2.imread(img2_path)

        # ensure the two images have the same dimensions
        image1_resized = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

        mse = np.mean((image1_resized - image2) ** 2)
        max_value = np.max(image1_resized)
        nrmse = np.sqrt(mse) / max_value
        ssim_val = ssim(image1_resized, image2, multichannel=True)
        psnr_val = psnr(image1_resized, image2, data_range=max_value)

        df_mse = df_mse.append({'Image1': images[i], 'Image2': images[j], 'MSE': mse}, ignore_index=True)
        df_nrmse = df_nrmse.append({'Image1': images[i], 'Image2': images[j], 'NRMSE': nrmse}, ignore_index=True)
        df_ssim = df_ssim.append({'Image1': images[i], 'Image2': images[j], 'SSIM': ssim_val}, ignore_index=True)
        df_psnr = df_psnr.append({'Image1': images[i], 'Image2': images[j], 'PSNR': psnr_val}, ignore_index=True)

# save the dataframes to csv
df_mse.to_csv('/content/drive/My Drive/TCC/images_red/mse_comparison.csv', index=False)
df_nrmse.to_csv('/content/drive/My Drive/TCC/images_red/nrmse_comparison.csv', index=False)
df_ssim.to_csv('/content/drive/My Drive/TCC/images_red/ssim_comparison.csv', index=False)
df_psnr.to_csv('/content/drive/My Drive/TCC/images_red/psnr_comparison.csv', index=False)


Streaming output truncated to the last 5000 lines.
64 586
64 587
64 588
64 589
64 590
64 591
64 592
64 593
64 594
64 595
64 596
64 597
64 598
64 599
64 600
64 601
64 602
64 603
64 604
64 605
64 606
64 607
64 608
64 609
64 610
64 611
64 612
64 613
64 614
64 615
64 616
64 617
64 618
64 619
64 620
64 621
64 622
64 623
64 624
64 625
64 626
64 627
64 628
64 629
64 630
64 631
64 632
64 633
64 634
64 635
64 636
64 637
64 638
64 639
64 640
64 641
64 642
64 643
64 644
64 645
64 646
64 647
64 648
64 649
64 650
64 651
64 652
64 653
64 654
64 655
64 656
64 657
64 658
64 659
64 660
64 661
64 662
64 663
64 664
64 665
64 666
64 667
64 668
64 669
64 670
64 671
64 672
64 673
64 674
64 675
64 676
64 677
64 678
64 679
64 680
64 681
65 66
65 67
65 68
65 69
65 70
65 71
65 72
65 73
65 74
65 75
65 76
65 77
65 78
65 79
65 80
65 81
65 82
65 83
65 84
65 85
65 86
65 87
65 88
65 89
65 90
65 91
65 92
65 93
65 94
65 95
65 96
65 97
65 98
65 99
65 100
65 101
65 102
65 103
65 104
65 105
65 106
65 107
65 108
65 109
65 

AttributeError: ignored

In [ ]:
a = spark.read.csv(f'/content/drive/My Drive/TCC/textOriginal.csv', inferSchema=True, header=True, sep=',')
a.filter("id=='UCJhKq7VQ0Ck6ffDOBA_GjUQ'").show()


+---+------------+-------+-------+
| id|textOriginal|videoId|replies|
+---+------------+-------+-------+
+---+------------+-------+-------+



In [ ]:
map = spark.read.csv(f'/content/drive/My Drive/TCC/images/image_map.csv', inferSchema=True, header=True, sep=';')

map.show(500, False)

+------------------------+------------+
|Channel ID              |Image Number|
+------------------------+------------+
|UC-Mbg_ogwCIYjviSBAW0Fkg|348         |
|UC-RLTPrTmGqS-JL-cphC7gw|349         |
|UC-l_6qwr4LXIWDkqisKZjkg|350         |
|UC-vR8Xaxv5OXsJ5fXn3fg8w|351         |
|UC-vqqCUNArWEvaDDvLAy4Kw|352         |
|UC0c7oH0oHFHjO1F65XkncTw|353         |
|UC0fFMbQa62krsR5Jc7nuNHA|354         |
|UC10Fl9OnP6L29z4lH_gwxOA|355         |
|UC10SH9hu6sxdwphpQUu00dA|356         |
|UC15Espd7t4_BXbDrpIP0abg|357         |
|UC17BBF8nGkUIHgq2-cKLTYA|358         |
|UC1SlCmy2pI1ohyyelu0Qv6w|359         |
|UC1ZyU_uI9LO7tO6_2c72a0A|360         |
|UC1dh1HIawM7Z4DN_oBCzmHw|361         |
|UC1ewfAPSOt6-5DGxIRkhwRg|362         |
|UC1nGyXySPO5JPU0bpVV49zA|363         |
|UC2-FjLJhovfUF5LPHvmSBaA|364         |
|UC29JmuJQTH2nkn7Y08GBZPg|365         |
|UC2PKYkT_vJ5drDSdM6Nf0cw|366         |
|UC2RotONc3X_nZBaKCEbN03Q|367         |
|UC2gyrBNuZ98y9i38YsA24OQ|368         |
|UC2iOu2wVx8LCAWHSA9AP2sw|369         |
